# Scraping political debates

Web scraping is a method that allows us to programmatically access and download the HTML underlying a website. It's the automated equivalent of manually clicking 'View Source' in most web browsers. 

Scraping is useful for grabbing large amounts of information from a site, but should be done ethically and responsibly. The UK Office for National Statistics has compiled a useful guide to ethical scraping (http://bit.ly/2IMCnJd) based on the following principles:

* Minimise burden on website owners
* Honour requests made by website owners to refrain from scraping their website
* Protect all personal data in all statistics and research outputs and seek ethical advice when scraping data that may identify individuals
* Apply scientific principles in the production of statistics and research based on web-scraped data and consider other sources of data
* Abide by all applicable legislation and monitor the evolving legal situation

### Visually inspecting the source code

Let's start by using our browser's 'Inspect' function to understand the Hansard website a bit better. 

*Navigate to https://hansard.parliament.uk/commons/ in your web browser and try to figure out:*

* *The full date range for which debate transcripts are available*
* *The URL pointing to debate transcripts for each day*


In [ ]:
import requests
import pandas as pd
import os
import time
from bs4 import BeautifulSoup
import json
import unidecode
import re

### Downloading raw HTML

Once we've figured this out, we can define a date range in pandas that spans the entire Hansard archive. To save time, let's limit today's analysis to 2018.

In [ ]:
hansard_date_range = [date.split(' ')[0] for date in pd.date_range(start='01/01/2018', end='12/31/2018').strftime('%Y-%m-%d')]
print(hansard_date_range[0:10])


In [ ]:
print(len(hansard_date_range))

The we can grab the raw HTML for every single day of debates from the very beginning of the Hansard archive to the present day. We introduce a time delay of 1 second between each download to be nice to the lovely folks at the Parliamentary Digital Service.
    

In [ ]:
os.mkdir('raw_hansard_files')
base_path = os.path.join(os.getcwd(),'raw_hansard_files')

In [ ]:
for date in hansard_date_range:
    
    date_url = 'https://hansard.parliament.uk/html/Commons/' + date + '/CommonsChamber'
    hansard_file = requests.get(date_url, allow_redirects=True)
    full_path = os.path.join(base_path,date+'.html')
    with open(full_path, 'wb') as f:
        f.write(hansard_file.content)
        
    with open('log.txt', 'a') as f:
        f.write(date)
        
    time.sleep(1)

Let's check how many files we've scraped

In [ ]:
n_scraped_files = len(os.listdir(base_path))
print(n_scraped_files)

### Removing invalid sessions

Not every date will have a valid sitting of Parliament. Inspect the contents of invalid dates with your text browser of choice and then programmatically remove them. 

In [ ]:
for raw_filepath in os.listdir(base_path):
    
    try:
        with open('raw_hansard_files/'+raw_filepath, 'r') as f:
            file_soup = BeautifulSoup(f.read(), 'html.parser')

        if str(file_soup)== 'The resource you are looking for has been removed, had its name changed, or is temporarily unavailable.':
            os.remove('raw_hansard_files/'+raw_filepath)
            print("Removed " + raw_filepath)
    except:
        raise
    

Let's check how many of the files we originally scraped are actually valid sittings of Parliament

In [ ]:
n_valid_files = len(os.listdir(base_path))
print(n_valid_files)

In [ ]:
n_valid_files/n_scraped_files


# Parsing and cleaning text

Now we've downloaded a set of files from Hansard, it's time to wrangle them into a format that's easy to analyse. This will be our first real encounter with the HTML parsing library Beautiful Soup, which takes messy HTML or XML and transforms it into an easily searchable tree. 

To get some practise in using Beautiful Soup, try the following brief exercises. Start by reading in one of our downloaded Hansard files.

In [ ]:
with open('raw_hansard_files/'+os.listdir(base_path)[0], 'r') as f:
    file_soup = BeautifulSoup(f.read(), 'html.parser')
        
print(file_soup)

BeautifulSoup lets us search this HTML for particular tags. To find all links, for example, we could run:


In [ ]:
all_links = file_soup.find_all('a')
print(all_links)

To find just the first link, we would run:

In [ ]:
first_link = file_soup.find('a')
print(first_link)

We can extract attributes from this result, such as the href and the text.

In [ ]:
print(first_link['href'])
print(first_link.get_text())

We can also test whether the result has particular attributes.

In [ ]:
first_link.has_attr('href')

In [ ]:
first_link.has_attr('id')

We can also narrow down our search according to the attributes of the HTML we're looking for

In [ ]:
file_soup.find_all('a',{'class':'nohighlight'})

When combined with list comprehensions, this gives us a powerful way of making sense of our HTML and pulling out relevant sections.

*As an exercise, modify this list comprehension to exclude 'Hon. Members' from the list of speakers.*

In [ ]:
[member_link.get_text() for member_link in file_soup.find_all('h2',{'class':'memberLink'})]

We want to represent each spoken contribution as a single observation or row in a huge Pandas DataFrame. The row for each spoken contribution should also contain columns giving:

* Contribution date
* Speaker name
* Speaker party 
* Speaker constituency 
* The title of the debate the speaker was contributing to
* Speaker gender 

*Open some of the downloaded HTML files in your web browser and use the 'Inspect' function to figure out which HTML tags, IDs and classes correspond uniquely to:*

* *Debate titles*
* *Speaker names*
* *Speaker ID numbers*

*Explore the JSON available at http://data.parliament.uk/membersdataplatform/services/mnis/members/query/House=Commons%7CIsEligible=true/ to figure out how we can use Speaker ID numbers to get information on speaker gender, party and constituency*


In [ ]:
os.mkdir('clean_hansard_files')

In [ ]:
contribution_counter = 0

for raw_filepath in [path for path in sorted(os.listdir(base_path)) if '.html' in path]:
    
    # Create an empty dataframe for this day's debates
    contributions_df = pd.DataFrame(columns=['date','debate_title','speaker_fullname','speaker_firstname','speaker_lastname','speaker_id','speaker_party','speaker_constituency','speaker_gender','contribution_text'])
    contributions_df.fillna('')
    
    with open('raw_hansard_files/'+raw_filepath, 'r') as f:
        file_soup = BeautifulSoup(f.read(), 'html.parser')
    
    date = raw_filepath[:-5]
    # Find all content items that contain individual debates
    debates = [content_item for content_item in file_soup.find_all('div',{'class':'content-item'}) if content_item.find('h2',{'class':'child-debate-title'})]

    # Loop through each content item and extract individual contributions
    for debate in debates:
        
        debate_title =  debate.find('h2',{'class':'child-debate-title'}).get_text()
        debate_contributions = [contribution for contribution in debate.find_all('div',{'class':'content-item'}) if (contribution.has_attr('id')) and ('contribution' in contribution['id'])]
        
        # Loop through each contribution and extract information about it
        for contribution in debate_contributions:
            
            speaker_info = contribution.find('h2',{'class':'memberLink'}).find('a')
            speaker_id = speaker_info['href'].split('=')[-1]
            
            if speaker_id != '0':
                
                try:
                    # Use the Parliamentary members' data platform to get more detailed information on the speaker
                    contribution_counter += 1
                    speaker_url = 'http://lda.data.parliament.uk/members/' + speaker_id + '.json'
                    speaker_json = json.loads(requests.get(speaker_url).content)['result']['primaryTopic']

                    contributions_df.at[contribution_counter,'speaker_id'] = speaker_id
                    contributions_df.at[contribution_counter,'date'] = date
                    contributions_df.at[contribution_counter,'debate_title'] = debate_title
                    contributions_df.at[contribution_counter,'speaker_firstname'] = unidecode.unidecode(speaker_json['givenName']['_value'])
                    contributions_df.at[contribution_counter,'speaker_lastname'] = unidecode.unidecode(speaker_json['familyName']['_value'])
                    contributions_df.at[contribution_counter,'speaker_fullname'] = unidecode.unidecode(speaker_json['fullName']['_value'])
                    contributions_df.at[contribution_counter,'speaker_gender'] = speaker_json['gender']['_value']
                    contributions_df.at[contribution_counter,'speaker_party'] = speaker_json['party']['_value']
                    contributions_df.at[contribution_counter,'speaker_constituency'] = speaker_json['constituency']['label']['_value']

                    # Get the text of the contribution and strip out whitespace characters
                    contributions_df.at[contribution_counter,'contribution_text'] = unidecode.unidecode(contribution.find('div',{'class':['contribution','content-container']}).get_text().strip())
                
                except:
                    pass
    
    contributions_df.to_csv('clean_hansard_files/'+date+'.csv',encoding='utf-8',index=False)
    print('Parsed: '+raw_filepath)


# Analysing language 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import string

from collections import Counter
import collections

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

Now let's explore our dataset. Let's start by reading in all our clean CSV files and aggregating into one dataframe.

In [ ]:
for idx, clean_filepath in enumerate(['clean_hansard_files/'+path for path in sorted(os.listdir('clean_hansard_files')) if '.csv' in path]):

    if idx==0:
        all_contributions_df = pd.read_csv(clean_filepath)
    else:
        all_contributions_df = all_contributions_df.append(pd.read_csv(clean_filepath))


We've got over 93000 speeches to analyse

In [ ]:
all_contributions_df.shape[0]

## What proportion of contributions are made by women?

Things to consider: 

* Is it fair to include female cabinet ministers (including the Prime Minister) in this analysis? 
* How should a 'contribution' be defined? How can we account for contribution length?
* Are women interrupted more than men? How are interruptions marked in Hansard?
* Is the proportion of women's speaking time equal to the proportion of women MPs?


## Have MPs become more disorderly during 2018? 

Things to consider:

* How should we define disorderliness? By interruptions, by the number of times John Bercow says 'order' or by his proportional speaking time?
